In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt

import torch
import torchvision
from torchvision import datasets, transforms
import torch.nn as nn
from torch.utils.data import DataLoader , ConcatDataset , Dataset
from torch.utils.data.sampler import SubsetRandomSampler
import torch.nn.functional as F
from PIL import Image

import os
print(os.listdir("/kaggle/input/cell-images-for-detecting-malaria/cell_images/cell_images"))

['Uninfected', 'Parasitized']


In [3]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [4]:
train_transforms = transforms.Compose([
    transforms.Resize((120, 120)),
    transforms.ToTensor(), 
    transforms.Normalize([0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

In [5]:
image_dir = "/kaggle/input/cell-images-for-detecting-malaria/cell_images/cell_images"
train_set = datasets.ImageFolder(image_dir, transform=train_transforms)

In [6]:
test_size = 0.2
num_train = len(train_set)
indices = list(range(num_train))
np.random.shuffle(indices)
test_split = int(np.floor(test_size * num_train))
test_index, train_index = indices[:test_split], indices[test_split:]

train_sampler = SubsetRandomSampler(train_index)
test_sampler = SubsetRandomSampler(test_index)

train_loader = DataLoader(train_set, sampler=train_sampler, batch_size=104)
test_loader = DataLoader(train_set, sampler=test_sampler, batch_size=58)

In [7]:
class Generator(nn.Module):
    def __init__(self, latent_dim, img_shape):
        super(Generator, self).__init__()

        self.model = nn.Sequential(
            nn.Linear(latent_dim, 128),
            nn.ReLU(),
            nn.Linear(128, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Linear(256, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(),
            nn.Linear(512, 1024),
            nn.BatchNorm1d(1024),
            nn.ReLU(),
            nn.Linear(1024, np.prod(img_shape)),
            nn.Tanh()
        )

    def forward(self, z):
        img = self.model(z)
        img = img.view(img.size(0), *img_shape)
        return img

In [8]:
class Discriminator(nn.Module):
    def __init__(self, img_shape):
        super(Discriminator, self).__init__()

        self.model = nn.Sequential(
            nn.Linear(np.prod(img_shape), 512),
            nn.LeakyReLU(0.2),
            nn.Linear(512, 256),
            nn.BatchNorm1d(256),
            nn.LeakyReLU(0.2),
            nn.Linear(256, 1),
            nn.Sigmoid()
        )

    def forward(self, img):
        img_flat = img.view(img.size(0), -1)
        validity = self.model(img_flat)
        return validity

In [9]:
class GAN(nn.Module):
    def __init__(self, generator, discriminator):
        super(GAN, self).__init__()
        self.generator = generator
        self.discriminator = discriminator

    def forward(self, z):
        img = self.generator(z)
        validity = self.discriminator(img)
        return validity

In [10]:
import torch.optim as optim
latent_dim = 128
img_shape = (3, 120, 120)
generator = Generator(latent_dim, img_shape).to(device)
discriminator = Discriminator(img_shape).to(device)
gan = GAN(generator, discriminator)
adversarial_loss = nn.BCELoss()

num_epochs_gan = 3
learning_rate_gan = 0.0002
learning_rate_classifier = 0.001

batch_size_gan = 64
criterion = nn.BCELoss()
optimizer_gan = optim.Adam(generator.parameters(), lr=0.0002, betas=(0.5, 0.999))
optimizer_D =  optim.Adam(discriminator.parameters(), lr=0.0002, betas=(0.5, 0.999))
generated_images_dir = "/kaggle/working/generated_images"

In [11]:
if not os.path.exists(generated_images_dir):
    os.makedirs(generated_images_dir)


In [12]:
def save_generated_images(images, epoch):
    for i, image in enumerate(images):
        save_path = os.path.join(generated_images_dir, f"generated_image_epoch_{epoch}_index_{i}.png")
        torchvision.utils.save_image(image, save_path)


In [13]:
class MosquitoNet(nn.Module):
    
    def __init__(self):
        super(MosquitoNet, self).__init__()
        
        self.layer1 = nn.Sequential(
            nn.Conv2d(3, 16, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        
        self.layer2 = nn.Sequential(
            nn.Conv2d(16, 32, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        
        self.layer3 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
            
        self.fc1 = nn.Linear(64*15*15, 512)
        self.fc2 = nn.Linear(512, 128)
        self.fc3 = nn.Linear(128, 2)
        self.drop = nn.Dropout2d(0.2)
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = out.view(out.size(0), -1)    # flatten out a input for Dense Layer
        out = self.fc1(out)
        out = F.relu(out)
        out = self.drop(out)
        out = self.fc2(out)
        out = F.relu(out)
        out = self.drop(out)
        out = self.fc3(out)
        
        return out

In [15]:
class GeneratedDataset(torch.utils.data.Dataset):
    def __init__(self, generated_dir, transform=None):
        self.generated_dir = generated_dir
        self.generated_files = os.listdir(generated_dir)
        self.transform = transform

    def __getitem__(self, index):
        img_path = os.path.join(self.generated_dir, self.generated_files[index])
        img = Image.open(img_path)
        if self.transform:
            img = self.transform(img)
        return img , 0

    def __len__(self):
        return len(self.generated_files)

In [16]:
num_generated_images = 0
for epoch in range(num_epochs_gan):
    for i,(imgs,_) in enumerate(train_loader, 0):
        valid = torch.ones(imgs.size(0), 1).to(device)
        fake = torch.zeros(imgs.size(0), 1).to(device)

        # Configure input
        real_imgs = imgs.to(device)

        # -----------------
        #  Train Generator
        # -----------------

        optimizer_gan.zero_grad()

        # Sample noise as generator input
        z = torch.randn(imgs.size(0), latent_dim).to(device)

        # Generate a batch of images
        gen_imgs = generator(z)
        save_generated_images(gen_imgs, epoch)

        # Loss measures generator's ability to fool the discriminator
        g_loss = adversarial_loss(discriminator(gen_imgs), valid)

        g_loss.backward()
        optimizer_gan.step()

        # ---------------------
        #  Train Discriminator
        # ---------------------

        optimizer_D.zero_grad()

        # Measure discriminator's ability to classify real from generated samples
        real_loss = adversarial_loss(discriminator(real_imgs), valid)
        fake_loss = adversarial_loss(discriminator(gen_imgs.detach()), fake)
        d_loss = (real_loss + fake_loss) / 2

        d_loss.backward()
        optimizer_D.step()


        

# Save the trained generator model

torch.save(generator.state_dict(), "/kaggle/working/model.pt")


In [17]:
generated_dataset = GeneratedDataset(generated_images_dir, transform=train_transforms)

# Check the length of the generated dataset
print("Length of generated dataset:", len(generated_dataset))

generated_loader = DataLoader(generated_dataset, batch_size=104, shuffle=True)

Length of generated dataset: 312


In [18]:
class CombinedDataset(Dataset):
    def __init__(self, original_dataset, generated_dataset, transform=None):
        self.original_dataset = original_dataset
        self.generated_dataset = generated_dataset
        self.transform = transform

    def __getitem__(self, index):
        if index < len(self.original_dataset):
            original_item, original_label = self.original_dataset[index]
            if self.transform:
                original_item = self.transform(original_item)
            return original_item, original_label
        else:
            generated_item, _ = self.generated_dataset[index - len(self.original_dataset)]
            if self.transform:
                generated_item = self.transform(generated_item)
            return generated_item, 0

    def __len__(self):
        return len(self.original_dataset) + len(self.generated_dataset)

In [19]:
combined_dataset = CombinedDataset(train_set, generated_dataset)

# Create a combined data loader
combined_loader = DataLoader(combined_dataset, batch_size=batch_size_gan, shuffle=True)

In [21]:
correct = 0
total = 0
class_total = [0 for _ in range(2)]
class_correct = [0 for _ in range(2)]
batch_size = 58

# Lists used in Confusion Matrix
actual = []
predict = []

In [22]:
model = MosquitoNet()
model.to(device)

MosquitoNet(
  (layer1): Sequential(
    (0): Conv2d(3, 16, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer2): Sequential(
    (0): Conv2d(16, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer3): Sequential(
    (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc1): Linear(in_features=14400, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=12

In [23]:
error = nn.CrossEntropyLoss()
learning_rate = 0.001
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
num_epochs = 20
batch_size = 100

In [24]:
train_losses = []
train_accuracies = []
for epoch in range(num_epochs):
    train_loss = 0.
    model.train()    # explicitly stating the training
    
    for i, (images, labels) in enumerate(combined_loader):
        images, labels = images.to(device), labels.to(device)
        train = images.view(-1, 3, 120, 120)
        outputs = model(train)
        
        optimizer.zero_grad()
        loss = error(outputs, labels)
        loss.backward()    # back-propagation
        optimizer.step()
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
     
        accuracy = correct / total
        train_losses.append(train_loss / len(train_loader.dataset))
        train_accuracies.append(accuracy)
     
    

# Save the trained model
torch.save(model.state_dict(), "/kaggle/working/modeltwo.pt")

/opt/conda/lib/python3.10/site-packages/torch/nn/functional.py:1345: UserWarning: dropout2d: Received a 2-D input to dropout2d, which is deprecated and will result in an error in a future release. To retain the behavior and silence this warning, please use dropout instead. Note that dropout2d exists to provide channel-wise dropout on inputs with 2 spatial dimensions, a channel dimension, and an optional batch dimension (i.e. 3D or 4D inputs).
  warnings.warn(warn_msg)


In [25]:
from sklearn.metrics import confusion_matrix, classification_report, f1_score

correct = 0
total = 0
class_total = [0 for _ in range(2)]
class_correct = [0 for _ in range(2)]
batch_size = 58

# Lists used in Confusion Matrix
actual = []
predict = []

model.eval()    # explicitly stating the testing 
with torch.no_grad():
    for i, (images, labels) in enumerate(combined_loader):
        images, labels = images.to(device), labels.to(device)
    
        actual.append(labels.data.tolist())
        test = images.view(-1, 3, 120, 120)
        outputs = model(test)
        predicted = torch.max(outputs, 1)[1]
        predict.append(predicted.data.tolist())
        total += len(labels)
        correct += (predicted == labels).sum().item()
        
        # Calculating classwise accuracy
        c = (predicted == labels).squeeze().bool()
        for pred, label in zip(predicted, labels):
            class_correct[label.item()] += (pred == label).item()
            class_total[label.item()] += 1
            actual_flat = [item for sublist in actual for item in sublist]
            predict_flat = [item for sublist in predict for item in sublist]

conf_matrix = confusion_matrix(actual_flat, predict_flat)

# Compute F1 score
f1 = f1_score(actual_flat, predict_flat, average='weighted')


# Generate classification report
class_report = classification_report(actual_flat, predict_flat)                
print("Accuracy on the Test set: {:.2f}%".format(correct * 100 / total))
print("Confusion Matrix:")
print(conf_matrix)
print("\nF1 Score:", f1)
print("\nClassification Report:")
print(class_report)

/opt/conda/lib/python3.10/site-packages/torch/nn/functional.py:1345: UserWarning: dropout2d: Received a 2-D input to dropout2d, which is deprecated and will result in an error in a future release. To retain the behavior and silence this warning, please use dropout instead. Note that dropout2d exists to provide channel-wise dropout on inputs with 2 spatial dimensions, a channel dimension, and an optional batch dimension (i.e. 3D or 4D inputs).
  warnings.warn(warn_msg)


Accuracy on the Test set: 99.52%
Confusion Matrix:
[[14022    69]
 [   66 13713]]

F1 Score: 0.9951560875901019

Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     14091
           1       0.99      1.00      1.00     13779

    accuracy                           1.00     27870
   macro avg       1.00      1.00      1.00     27870
weighted avg       1.00      1.00      1.00     27870

